# Create Minds Matter Calendar events 

This notebook will read a computer-readable google sheet of calendar events and programmatically create each event in the Google calendar of the Minds Matter Admin account. It will also invite the relevant groups to each event. 

**Warning: You must complete the pre-work in the [How to run the calendar script](https://docs.google.com/document/d/10Kf_lHase22wk-F24BV5QlQ2uinig5b5E7TwfCtG1wI/edit?usp=sharing) google doc before you can run this notebook!!**

If you've completed the pre-work, proceed to filling in the noteobook inputs and running the cells:

### Step 1: Fill in manual inputs

In [ ]:
## FILL IN MANUAL INPUTS ##

# copy & paste the Spreadsheet ID from the pre-work steps here
SPREADSHEET_ID = "1jy2kZF4rHHUcGEzYcZfA6kukUdqaSj7FWzKIoyoapcM"

In [ ]:
from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import numpy as np
import os.path
import pickle
import time

import pandas as pd

from helper_functions import * 

In [ ]:
%load_ext nb_black

In [ ]:
print(f"This notebook was run on {datetime.datetime.today()}")

### Step 2: Authenticate your account

Running the following cells will result in a pop-up window allowing you to authenticate and store the credentials in a `token.pickle` file. A "warning" may pop up telling you that Google hasn't verified this app. Click "Advanced" and then "Go to Untitled Project." Select all of the items that you need to access and click "Continue."

Once you have finished authenticating, a `token.pickle` file will get dropped into your working directory.

In [ ]:
creds = read_credentials()

In [ ]:
# build the calendar service
service = build("calendar", "v3", credentials=creds)

Test that the authentication has worked by call the Calendar API and printing the next 10 events in the user's calendar. Note: You may not have any upcoming events, that is fine. If this cell runs successfully, the Google API is working as intended. 

In [ ]:
now = datetime.datetime.utcnow().isoformat() + "Z" # 'Z' indicates UTC time
print('Getting the upcoming 10 events')
events_result = service.events().list(calendarId=CALENDER_ID, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    start = event['start'].get('dateTime', event['start'].get('date'))
    print(start, event['summary'])

### 3. Read data from the Academic calendar--Computer readable Google sheet

In [ ]:
df = read_google_sheet(SPREADSHEET_ID, creds=creds)
print(f"Loading {df.shape[0]} rows.")

In [ ]:
# sanity check
df.head()

### 4. Create events

Note this will take a few minutes!

In [ ]:
eventservice=service.events()

In [ ]:
# create events
for k,row in df.iterrows():
    body={
        "description": row['Event Title'],
        "summary":  row['Event Title'],
        "start":{
            "dateTime": row.starttime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "end":{
            "dateTime": row.endtime.isoformat(),
            "timeZone": "America/Los_Angeles"
        },
        "colorId": f"{row.Color}",
        "guestsCanInviteOthers": False

    }
    
    if row.calendarEventID:
        pass
    else:
        r=eventservice.insert(calendarId=calendarID,
                             body=body)
        response=r.execute()
        df.loc[k, 'calendarEventID']=response["id"]
        time.sleep(.2)

Spot check that events were created by going into the Admin Google Calendar and checking a few to see if they show up.

### 5. Invite user groups to events

Note this will take a few minutes!

In [ ]:
for k,row in df.iterrows():
    emails=map(lambda x: x.replace(' ','') ,row['Share with group'].split(','))
    event = eventservice.get(calendarId=calendarID, eventId=row.calendarEventID).execute()
    attendees=event.get('attendees',[])
    for email in emails:
        try:
            invite=next(a for a in attendees if a['email']==email + "@mindsmatterseattle.org")
        except StopIteration:
            print(f'inviting {email} to {event["summary"]}')
            invite = {
                'email': f'{email}@mindsmatterseattle.org'
            }
            attendees.append(invite)

    event['attendees'] =  attendees 
    eventservice.update(calendarId=calendarID, eventId=row.calendarEventID, body=event).execute()

### 6 (Optional): Save output for historical records

In [ ]:
df.to_csv("<ENTER_DESIRED_NAME>", index=False)